## Add radii to csv

This script adds the average radii for each participant to csv for further analysis (i.e. linear regression) using R.

In [9]:
import os

#set directory
print(os.getcwd())
os.chdir('C:/Users/sijam/Documents/GitHub/Eye-Tracking-Neural-Sys')

#Import all functions 
from pygazeanalyser_functions import *

#import csv library and pandas
import csv
import pandas as pd
import matplotlib

C:\Users\sijam\Documents\GitHub\Eye-Tracking-Neural-Sys


In [27]:
#participant details

PIDs = ["002","003","004","005","006","007","008","009","010",
        "011","012","013","014","015","016","017","018","019","020",
        "021","022","023","024","025","026","027","028","029","030",
        "031","032","033","034","035","036"] #all the participant IDs
PIDs = ["021","022","023","024","025","026","027","028","029","030",
        "031","032","033","034","035","036"]
game_nrs  = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16] #game numbers
trial_nrs = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15] #trial 16 and 17 are "out of index"
#phase = 'fixation'
phase = 'stimulus'
lenData = 35 #we have 35 participants
dataCol = 1 #coloumn nr of "Delta_radius" in the new csv file, actually 2, but python starts with index 0

#if the csv file doesn't exist yet:
#with open("foo.csv", "w") as my_empty_csv:
#  # now you have an empty file
#  pass  # or write something to it

#"{0}\\Data File\\results_data_subjects_V2.csv".format(os.getcwd())
df = pd.read_csv("C:/Users/sijam/Documents/GitHub/Eye-Tracking-Neural-Sys/Data Files/results_data_subjects_V2.csv") #this is the output csv for delta mean radius by subject
df_original = pd.read_csv("C:/Users/sijam/Documents/GitHub/Eye-Tracking-Neural-Sys/Data Files/noise_data_trials.csv") #this is the input csv containing information about the noise
#print(df)
#print(df_original)

radius_delta = df['delta_radius_stimulation']
radius_low = df['radius_low_stimulation']
radius_high = df['radius_high_stimulation']
noise_list = df_original['perc_noise']

radius_delta = radius_delta.values.tolist()
radius_low = radius_low.values.tolist()
radius_high = radius_high.values.tolist()
noise_list = noise_list.values.tolist()
print("delta list: ", radius_delta)
print("high list: ", radius_low)
print("low list: ", radius_high)

nan = radius_delta[34] #save nan as nan, only if the delta_radii in the output was empty in the beginning
#nan = "nan"

delta list:  [-1.42578125, -0.25390625, -2.9375, 4.48046875, 2.3203125, 0.66015625, -0.328125, -5.5, -2.5390625, -3.42578125, 1.9375, 2.08984375, -2.33203125, 1.18359375, 3.33984375, 0.28125, -0.03125, 4.47265625, -4.4921875, 0.69140625, 0.5625, -0.73046875, 1.4296875, -2.5078125, 0.421875, -0.23828125, -2.3671875, 0.1796875, 2.40625, -1.45703125, 4.34765625, 2.87890625, 1.82421875, 1.75, 1.18359375]
high list:  [14.04296875, 10.39453125, 9.4375, 9.55078125, 0.0, 17.4296875, 17.98046875, 8.875, 7.6953125, 14.42578125, 17.6953125, 11.46484375, 10.9140625, 9.87109375, 12.5703125, 19.33984375, 23.48828125, 8.9609375, 17.38671875, 7.46484375, 14.18359375, 5.27734375, 6.3828125, 6.41015625, 0.421875, 4.4609375, 5.875, 3.8359375, 2.7890625, 22.93359375, 10.625, 11.52734375, 7.82421875, 10.84765625, 17.24609375]
low list:  [12.6171875, 10.140625, 6.5, 14.03125, 2.3203125, 18.08984375, 17.65234375, 3.375, 5.15625, 11.0, 19.6328125, 13.5546875, 8.58203125, 11.0546875, 15.91015625, 19.62109375, 

In [26]:
for PID_string in (PIDs):

    PID_int = int(PID_string)
    PID = "G"+PID_string
    z0 = 0 #counts the low noise mean radius of each game within a subject
    z1 = 0 #counts the high noise mean radius of each game within a subject
    for game_nr in (game_nrs):
        data = read_data(PID, game_nr, phase)
        #print("Data: ", data) #Uncomment this line to print
        y0 = 0 # counts the mean radii of each trial within a game when you have low noise
        y1 = 0 # counts the mean radii of each trial within a game when you have high noise
        for trial_nr in (trial_nrs):
            #draw_heatmap_trial(data,PID,game_nr,trial_nr) #if the heatmap don't exist yet
            image_name = 'output_images/{}_{}_{}_heatmap.png'.format(PID, game_nr, trial_nr) # name of the image you want to load
            #print(image_name)
            centers,radii,image_out = return_centroid_heatmap(image_name)
            matplotlib.pyplot.close()
            #print("Radii: ", radii)
            if len(radii) == 0:
                radii = nan
            elif len(radii) >= 2:
                x = 0 #counts each radii
                for i in range(0,len(radii)):
                    x = x+radii[i]
                radii = [x/len(radii)]
            #print("Radii: ", radii)
            dataRowNoise = 1 + (PID_int-2)*(18*16) + (game_nr-1)*18 + trial_nr
            if not type(radii) == float:
                if noise_list[dataRowNoise] == 6:
                    y0 = y0 + radii[0] #low noise
                elif noise_list[dataRowNoise] == 18:
                    y1 = y1 + radii[0] #high noise
        y0 = y0/len(trial_nrs) #this is the average radius for a low noise game
        y1 = y1/len(trial_nrs) #this is the average radius for a high noise game
        #y = y1-y0 #delta value
        #dataRow2 = 1 + (PID_int-2)*(16) + game_nr #row nr in the csv file for games
        #radii_list2[dataRow2] = y
        z0 = z0+y0 #low noise
        z1 = z1+y1 #high noise
        print(PID_int, " game ", game_nr, " done!")
    z0 = z0/len(game_nrs)
    z1 = z1/len(game_nrs)
    z=z1-z0 #delta value
    dataRow = PID_int-2 #row nr in the csv file for games
    radius_high[dataRow] = z1
    radius_low[dataRow]= z0
    radius_delta[dataRow] = z #high noise game - low noise game
    df['delta_radius_stimulation'] = radius_delta
    df['radius_low_stimulation'] = radius_low
    df['radius_high_stimulation'] = radius_high
    df.to_csv("C:/Users/sijam/Documents/GitHub/Eye-Tracking-Neural-Sys/Data Files/results_data_subjects_V2.csv", index=False)

#df2['delta_radii'] = radii_list2
#df2.to_csv("results_data_games_V1.csv", index=False)
print("DONE")

21  game  1  done!
21  game  2  done!
21  game  3  done!
21  game  4  done!
21  game  5  done!
21  game  6  done!
21  game  7  done!
21  game  8  done!
21  game  9  done!
21  game  10  done!
21  game  11  done!
21  game  12  done!
21  game  13  done!
21  game  14  done!
21  game  15  done!
21  game  16  done!
22  game  1  done!
22  game  2  done!
22  game  3  done!
22  game  4  done!
22  game  5  done!
22  game  6  done!
22  game  7  done!
22  game  8  done!
22  game  9  done!
22  game  10  done!
22  game  11  done!
22  game  12  done!
22  game  13  done!
22  game  14  done!
22  game  15  done!
22  game  16  done!
23  game  1  done!
23  game  2  done!
23  game  3  done!
23  game  4  done!
23  game  5  done!
23  game  6  done!
23  game  7  done!
23  game  8  done!
23  game  9  done!
23  game  10  done!
23  game  11  done!
23  game  12  done!
23  game  13  done!
23  game  14  done!
23  game  15  done!
23  game  16  done!
24  game  1  done!
24  game  2  done!
24  game  3  done!
24  game  